In [ ]:
#This file contains code for analysing and organizing datasets belonging to Sui Northern Gas Pipelines Limited. Specifically, it id designed to analyse EVC (Electronic Gas Volume Corrector) Data.
#Each dataset contains thousands of rows (the data is hourly) and has 25+ columns where each column is a parameter relating to the EVCs. Example parameters include minimum pressure, maximum pressure, flow, teperature, battery volatage, time, and more.
#Each of the below functions has its own purpose. Each function organizes the dataset according to some scrutiny point (e.g. any points with 0 pressure, or any points with 0 or reduced flow).
#After finding all rows that satisfy that criteria, the function creates a CSV file containing rows that pertain to that scrutiny point from the larger dataset.
#Each function also saves the resultant rows to a new and smaller file. So at the end of the program, each function will write to a csv file inside a folder called scrutiny files.
#At the end the program compiles the results of all the scrutiny points into a table so a user may see which scrutiny points have the greatest number of concerned rows and are thus deserving of more attention.
#Apart from inputting the inputs for each function through the command line interface, this program makes use of a config file using config parser to allow the user to enter the relevant inputs.
#Import statement to use pandas library and associated function
#Filepath to read csv file of focus
import configparser
import pandas as pd
import matplotlib.pyplot as plt
from numpy import average
# file_import = input('Input CSV File. E.g. Meter Data.csv')
# if file_import:
#     filepath = file_import
# else:
filepath = 'Excel SNGPL data.csv'
filepather = 'SNGPL Excel Dual Data.csv'
data_path = 'Dual Sensor Absolute.csv'
doc = pd.read_csv(filepath)
config = configparser.ConfigParser()
config.read('configfile.ini')
worker = pd.DataFrame({'DateTime':['ZERO PRESS SCRUTINY'], 'Pressure p1':['ZERO PRESS SCRUTINY'], 'Battery voltage':['ZERO PRESS SCRUTINY'], 'Battery capacity':['ZERO PRESS SCRUTINY'], 'Abs. pressure p_abs':['ZERO PRESS SCRUTINY'], 'Internal temp.':['ZERO PRESS SCRUTINY'], 'Spare temp. t_sp':['ZERO PRESS SCRUTINY'], 'p1min':['ZERO PRESS SCRUTINY'],
 'T_p1min':['ZERO PRESS SCRUTINY'], 'p1max':['ZERO PRESS SCRUTINY'], 'T_p1max':['ZERO PRESS SCRUTINY'], 'Primary volume V1':['ZERO PRESS SCRUTINY'], 'D_Primary volume V1':['ZERO PRESS SCRUTINY'], 'Spare prim. vol. Vs1':['ZERO PRESS SCRUTINY'], 'D_Spare prim. vol. Vs1':['ZERO PRESS SCRUTINY'], 
 'Base volume Vb1':['ZERO PRESS SCRUTINY'], 'D_Base volume Vb1':['ZERO PRESS SCRUTINY'], 'Spare base vol. Vbs1':['ZERO PRESS SCRUTINY'], 'D_Spare base vol. Vbs1':['ZERO PRESS SCRUTINY'], 'Flow Q1':['ZERO PRESS SCRUTINY'], 'Q1max':['ZERO PRESS SCRUTINY'], 'T_Q1max':['ZERO PRESS SCRUTINY'], 'Base flow Qb1':['ZERO PRESS SCRUTINY'], 
 'Convers. factor C1':['ZERO PRESS SCRUTINY'], 'Comp. ratio Z/Zb K1':['ZERO PRESS SCRUTINY'], 'Status':['ZERO PRESS SCRUTINY'], 'Tamper':['ZERO PRESS SCRUTINY'], 'Battery state':['ZERO PRESS SCRUTINY'], 'p1_min':['ZERO PRESS SCRUTINY'], 'p1_max':['ZERO PRESS SCRUTINY'], 't1_min':['ZERO PRESS SCRUTINY'], 't1_max':['ZERO PRESS SCRUTINY'], 'Q1_min':['ZERO PRESS SCRUTINY'], 'Q1_max':['ZERO PRESS SCRUTINY']})

docs = pd.read_csv(filepath, header = 0, infer_datetime_format = True)
pd.options.display.max_columns = 100
doc['DateTime'] = pd.to_datetime(doc['DateTime'], infer_datetime_format=True)
# The above reads in the csv file, changes VS code settings to preview 100 columns at a time, and converts the column 'DateTime' from a string datatype to pd.to_datetime datatype
# The below is the start of the functions. All functions can take an argument when calling to avoid input statements, E.g. zero_p1(15).
# All functions return a slice of the original dataframe according to the scrutiny conditions. All inputs are converted to floats before comparison or condition checking.
# All functions have a default value so if no arguments/inputs are specified, then the result will take hold with a default value.
# Each scrutiny point saves itself as a CSV in the same folder for further detail.

datatypes = {1: 'Elgas Abs Single', 2: 'Elgas Abs Dual', 3: 'Elgas Gauge', 4: 'RFLO Abs Single'}
datatype_input = input('Please input the type of data file you are inputting. Enter {key_1} for {value_1}, {key_2} for {value_2}, {key_3} for {value_3}, or {key_4} for {value_4}.'.format(key_1 = list(datatypes.keys())[0], value_1 = datatypes[1], 
key_2 = list(datatypes.keys())[1], value_2 = datatypes[2], key_3 = list(datatypes.keys())[2], value_3 = datatypes[3], key_4 = list(datatypes.keys())[3], value_4 = datatypes[4]))
print(datatypes[int(datatype_input)])

In [ ]:
#The below function is for dual sensor data and looks for 0 pressure in both sensors.
def zero_p1_p2_dual(press_input = 15, datatype = datatype_input):
    if datatype != '2':
        return "File does not contain dual sensing data, E.g. p2."
    if not press_input:
        press_input = input('Please input zero limit for pressure parameters. Default is 15.')
        if not press_input:
            press_input = 15
    test = doc[(doc['Pressure p1'] < float(press_input)) & (doc['Pressure p2'] < float(press_input))]
    test.to_csv('Scrutiny_Files/Dual Zero P1 P2 Scrutiny.csv')
    return test
#The below function looks for 0 MINIMUM pressure in both sensors.
def zero_min_p1_p2_dual(press_input = 15, datatype = datatype_input):
    if datatype != '2':
        return "File does not contain dual sensing data, E.g. p2."
    if not press_input:
        press_input = input('Please input zero limit for minimum pressure parameters. Default is 15.')
        if not press_input:
            press_input = 15
    test = doc[(doc['p1min'] < press_input) & (doc['p2 min'] < press_input)]
    test.to_csv('Scrutiny_Files/Dual Zero Min P1 P2 Scrutiny.csv')
    return test
#The below function looks for a pressure differential between the 2 sensors that is greater than a specific input.
def dual_pressure_differential(press_diff_input = 10, datatype = datatype_input):
    if datatype != '2':
        return "File does not contain dual sensing data, E.g. p2."
    test = doc[(abs(doc['p1min'] - doc['p2 min']) > float(press_diff_input))]
    test.to_csv('Scrutiny_Files/Dual Pressure Differential Scrutiny.csv')
    return test
#The below function looks for a pressure differential between the 2 sensors that is greater than a specific input, like the function above, whilst also looking for simultaneous 0 flow.
def dual_pressure_diff_flow(press_diff_input = 10, flow_lower = 2000, flow_upper = 5000, datatype = datatype_input):
    if datatype != '2':
        return 'File does not contain dual sensing data, E.g. p2.'
    test = doc[((abs(docs['p1min'] - doc['p2 min']) > float(press_diff_input)) & 
    ((doc['D_Primary volume V1'] == 0)))]
    test.to_csv('Scrutiny_Files/Dual Pressure Differential Flow Scrutiny.csv')
    return test
#The below looks for 0 pressure in sensor 1.
def zero_p1(press_input = 15, datatype = datatype_input):
    if datatype == '2' or datatype == '1':
        test = doc[(doc['p1max']<float(press_input)) & (doc['D_Primary volume V1'] > 0)]
    elif datatype == '3':
        test = doc[(doc['p1max']< 0.35) & (doc['D_Primary volume V1'] > 0)]
    test.to_csv('Scrutiny_Files/Zero P1 Scrutiny.csv')
    return test
#The below looks for 0 MINIMUM pressure in sensor 1.
def zero_p1min(min_press_input = 15, datatype = datatype_input):
    if datatype == '2' or datatype == '1':
        test = doc[(doc['p1min']<float(min_press_input)) & (doc['p1max'] > float(min_press_input)) & (doc['D_Primary volume V1'] > 0)]
    elif datatype == '3':
        test = doc[(doc['p1min']< 0.35) & (doc['p1max'] > 0.35) & (doc['D_Primary volume V1'] > 0)]
    test.to_csv('Scrutiny_Files/Zero P1 Min Scrutiny.csv')
    return test

#The functions below use & bitwise operators to check for simultaneous conditions.
#The below looks for both 0 pressure in sensor 1 and 0 flow.
def zero_p1_flow(press_input = 15, datatype = datatype_input):
    if datatype == '2' or datatype == '1':
        test = doc[(doc['p1max'] < float(press_input)) & (doc['D_Primary volume V1'] == 0)]
    elif datatype == '3':
        test = doc[(doc['Abs. pressure p_abs'] < 0.35) & (doc['D_Primary volume V1'] == 0)]
    test.to_csv('Scrutiny_Files/Zero P1 Flow.csv')
    return test
#The below looks for both 0 MINIMUM pressure in sensor 1 and 0 flow.
def zero_p1min_flow(min_press_input = 15, datatype = datatype_input):
    if datatype == '2' or datatype == '1':
        test = doc[(doc['p1min']<float(min_press_input)) & (doc['p1max'] > float(min_press_input)) & (doc['D_Primary volume V1'] == 0)]
    elif datatype == '3':
        test = doc[(doc['p1min']< 0.35) & (doc['p1max'] > 0.35) & (doc['D_Primary volume V1'] == 0)]
    test.to_csv('Scrutiny_Files/Zero P1 Min Flow.csv')
    return test
#The below looks for points that are representative of 0 flow.
def zero_flow(datatype = datatype_input):
    if datatype == '2' or datatype == '1':
        test = doc[(doc['p1min'] > 15) & (doc['D_Primary volume V1'] == 0)]
    if datatype == '3':
        test = doc[(doc['p1min'] > 0.35) & (doc['D_Primary volume V1'] == 0)]
    test.to_csv('Scrutiny Files/Zero Flow.csv')
    return test
#The below looks for rows with reduced flow.
def reduced_flow(rated_capacity = 5000):
    test = doc[(doc['D_Primary volume V1']<(0.5*int(rated_capacity)))]
    if len(test) == 1:
        return "There is 1 row exhibiting reduced flow."
    else:
        return "There are {} rows exhibiting reduced flow.".format(len(test))

# Functions such as the one below use a combination of & and | bitwise operators to check for one definite condition and two alternative second conditions.
#The below looks for 0 pressur or MINIMUM pressure as well as 0 flow.
def zero_p1_or_p1min_and_flow(min_abs_press_input = None, datatype = datatype_input):
    if min_abs_press_input:
        if 'Pressure p2' in doc.columns:
            test = doc[(((doc['p1min']<float(min_abs_press_input)) & (doc['p2 min'] < float(min_abs_press_input)))| ((doc['Pressure p1'] < float(min_abs_press_input)) & (doc['Pressure p2'] < float(min_abs_press_input)))) & (doc['Flow Q1'] == 0)]
        else:
            test = doc[(doc['Flow Q1'] == 0) & ((doc['Abs. pressure p_abs']<float(min_abs_press_input)) |(doc['p1min']+14.65 <float(min_abs_press_input)))]
    else:
        min_abs_press_input = input("Please input the zero minimum/absolute pressure limit you would like to look below. If nothing is entered, default is 15. \n E.g. inputting 15 will return all rows where minimum/absolute pressure is below 15.")
        if min_abs_press_input:
            if 'Pressure p2' in doc.columns:
                test = doc[(((doc['p1min']<float(min_abs_press_input)) & (doc['p2 min'] < float(min_abs_press_input)))| ((doc['Pressure p1'] < float(min_abs_press_input)) & (doc['Pressure p2'] < float(min_abs_press_input)))) & (doc['Flow Q1'] == 0)]
            else:
                test = doc[(doc['Flow Q1'] == 0) & ((doc['Abs. pressure p_abs']<float(min_abs_press_input)) |(doc['p1min']+14.65 <float(min_abs_press_input)))]
        else:
            if 'Pressure p2' in doc.columns:
                test = doc[(((doc['p1min']<15) & (doc['p2 min'] < 15))| ((doc['Pressure p1'] < 15) & (doc['Pressure p2'] < 15))) & (doc['Flow Q1'] == 0)]
            else:
                test = doc[(doc['Flow Q1'] == 0) & ((doc['Abs. pressure p_abs']<15) |(doc['p1min']+14.65 <15))]
    test.to_csv('Scrutiny_Files/Zero Abs Min P1 & Flow.csv')
    return test
#The below looks for rows where flow is above the rated capacity.
def flow_above_rated_capacity(rated_capacity = 5000, datatype = datatype_input):
    if rated_capacity:
        test = doc[doc['Q1max']> float(rated_capacity)]
    else:
        rated_capacity = input("Please input the set meter rated capacity? E.g. 5000")
        if rated_capacity:
            test = doc[doc['Q1max']>float(rated_capacity)]
        else: 
            print('No set meter rated capacity inputted, will use default value of 5000')
            test = doc[doc['Q1max']>float(5000)]
    test.to_csv('Scrutiny_Files/Flow Above Capacity.csv')
    return test

#The below two functions have contingency statements in the case that only one of the two inputs are specified
#The below looks for rows that have both abnormal pressure and abnormal temperature values.
def abnormal_press_and_temp(temp_lower = None, temp_upper = None, press_input = None, datatype = datatype_input):
    if temp_lower and temp_upper and press_input:
        if 'Pressure p2' in doc.columns:
            test = doc[((doc['Pressure p1']<float(press_input)) & (doc['Pressure p2'] < float(press_input)))& ((doc['Internal temp.'] > float(temp_upper)) | (doc['Internal temp.'] < float(temp_lower)))]
        else:
            test = doc[(doc['Abs. pressure p_abs']<15) & ((doc['Internal temp.'] < float(temp_lower)) | (doc['Internal temp.'] > float(temp_upper)))]
    else:
        temp_lower = input("Please enter lower temperature limit, default value is 60.")
        temp_upper = input('Please input upper temperature limit, default is 140')
        press_input = input('Please input lower limit to look below for pressure values. Default is 15')
        if not press_input:
            press_input = 15
        if not temp_upper:
            temp_upper = 140
        if not temp_lower:
            temp_lower = 60
        if 'Pressure p2' in doc.columns:
            test = doc[((doc['Pressure p1']<float(press_input)) & (doc['Pressure p2'] < float(press_input)))& ((doc['Internal temp.'] < float(temp_lower)) | (doc['Internal temp.'] > float(temp_upper)))]
        else:
            test = doc[(doc['Abs. pressure p_abs']<float(press_input)) & ((doc['Internal temp.'] < float(temp_lower)) | (doc['Internal temp.'] > float(temp_upper)))]
    test.to_csv('Scrutiny_Files/Abnormal Press & Temp.csv')
    return test
#The below looks for rows with abnormal flow values.
def abnormal_flow(flow_lower = None, flow_upper = None, datatype = datatype_input):
    if flow_lower and flow_upper:
        test = doc[((doc['Flow Q1']<float(flow_lower)) | (doc['Flow Q1']>float(flow_upper)))]
    else:
        flow_upper = input('Please input an upper limit for normal flow values. Default value is 1700')
        flow_lower = input('Please input a lower limit for normal flow values. Default value is 850')
        if flow_upper and flow_lower:
            test = doc[((doc['Flow Q1']<float(flow_lower)) | (doc['Flow Q1']>float(flow_upper)))]
        elif flow_upper:
            test = doc[((doc['Flow Q1']<850) | (doc['Flow Q1']>float(flow_upper)))]
        elif flow_lower:
            test = doc[((doc['Flow Q1']<float(flow_lower)) | (doc['Flow Q1']>1700))]
        else: 
            print('No values inputted, default values of 850 and 1700 are used.')
            test = doc[((doc['Flow Q1']<850) | (doc['Flow Q1']>1700))]
    test.to_csv('Scrutiny_Files/Abnormal Flow.csv')
    return test
    # Email rizwan as there are no flow values within set limit of 850 - 1700. Almost all values are 0 or in the low 100s.
#The below looks for rows where flow is negative.
def reverse_flow(datatype = datatype_input):
    test = doc[doc['Flow Q1'] < 0]
    test.to_csv('Scrutiny_Files/Reverse Flow.csv')
    return test

#The below looks for rows where the accumulated flow is negative.
def accum_reverse_flow(datatype = datatype_input):
    test = doc[doc['D_Primary volume V1'] < 0]
    summed = 0
    for i in test.index:
        summed += int(test.at[i, 'D_Primary volume V1'])
    test.to_csv('Scrutiny_Files/Accumulated Reverse Flow.csv')
    return summed
#The below looks for rows where flow is reverse and there is 0 pressure or 0 MINIMUM pressure.
def rev_flow_and_zero_min_or_abs_press(min_abs_press_input = None, datatype = datatype_input):
    if min_abs_press_input:
        if 'Pressure p2' in doc.columns:
            test = doc[(((doc['p1min']<float(min_abs_press_input)) & (doc['p2 min']<float(min_abs_press_input)))| ((doc['Pressure p1'] < float(min_abs_press_input)) & (doc['Pressure p2'] < float(min_abs_press_input)))) & ((doc['Flow Q1'] < 0))]
        else:
            test = doc[(doc['Flow Q1'] < 0) & ((doc['Abs. pressure p_abs']<float(min_abs_press_input))|(doc['p1min']+14.65<float(min_abs_press_input)))]
    else:
        min_abs_press_input = input("Please input the zero minimum/absolute pressure limit you would like to look below. If nothing is entered, default is 15. \n E.g. inputting 15 will return all rows where minimum/absolute pressure is below 15.")
        if min_abs_press_input:
            if 'Pressure p2' in doc.columns:
                test = doc[(((doc['p1min']<float(min_abs_press_input)) & (doc['p2 min']<float(min_abs_press_input)))| ((doc['Pressure p1'] < float(min_abs_press_input)) & (doc['Pressure p2'] < float(min_abs_press_input)))) & ((doc['Flow Q1'] < 0))]
            else:
                test = doc[(doc['Flow Q1'] < 0) & ((doc['Abs. pressure p_abs']<float(min_abs_press_input))|(doc['p1min']+14.65<float(min_abs_press_input)))]
        else:
            if 'Pressure p2' in doc.columns:
                test = doc[(((doc['p1min']<15) & (doc['p2 min']<15))| ((doc['Pressure p1'] < 15) & (doc['Pressure p2'] < 15))) & ((doc['Flow Q1'] < 0))]
            else:
                test = doc[(doc['Flow Q1'] < 0) & ((doc['Abs. pressure p_abs']<15)|(doc['p1min']+14.65<15))]
    test.to_csv('Scrutiny_Files/Rev Flow & Zero Abs Min P1.csv')
    return test
#The below looks for rows where the battery voltage is below a specified limit.
def battery_variations(battery_lower = None, datatype = datatype_input):
    if battery_lower:
        test = doc[doc['Battery voltage']< float(battery_lower)]
    else:
        battery_lower = input('Please input a lower limit for acceptable battery voltage. Default is 2.5')
        if battery_lower:
            test = doc[(doc['Battery voltage']< float(battery_lower))]
        else:
            test = doc[doc['Battery voltage']< 2.5]
    test.to_csv('Scrutiny_Files/Battery Variations.csv')
    return test
#The below looks for rows where the corrected volume compatability factor (corrected volume/registered volume) is above a specified amount.
def reg_corr_volume_compat(init = None, datatype = datatype_input):
    if init:
        test = doc[((doc['Base volume Vb1'])/(doc['Primary volume V1']))> float(init)]
    else:
        init = input('Please input the minimum threshold to look above for (corrected volume/registered volume). E.g. if corrected = 2000, registered = 1500, then compat_ratio = 2000/1500 = 1.33. Default is 1.001')
        if init:
            test = doc[((doc['Base volume Vb1'])/(doc['Primary volume V1']))> float(init)]
        else:
            test = doc[(doc['Base volume Vb1']/doc['Primary volume V1']> float(1.001))]
    test.to_csv('Scrutiny_Files/Registered Corrected Compat Factor.csv')
    return test
#The below looks for rows where there is 0 or reduced flow and the tamper alarm is triggered.
def tamper_alarm_and_zero_reduced_flow(flow_lower = None, rated_capacity = 5000, datatype = datatype_input):
    if flow_lower:
        test = doc[(doc['Tamper'] == 1) & ((doc['Flow Q1'] == 0) | (doc['Flow Q1'] < float(flow_lower)))]
    else: 
        flow_lower = input('Please input the reduced flow lower value. Default is 850.')
        if flow_lower:
            test = doc[(doc['Tamper'] == 1) & ((doc['Flow Q1'] == 0) | (doc['Flow Q1'] < float(flow_lower)))]
        else:
            test = doc[(doc['Tamper'] == 1) & ((doc['Flow Q1'] == 0) | (doc['Flow Q1'] < 850))]
    test.to_csv('Scrutiny_Files/Tamper Alarm & Zero Reduced Flow.csv')
    return test
#The below looks for rows where the pressure 1 sensor is between a specified range.
def press_assortment_range(press_lower = None, press_upper = None, datatype = datatype_input):
    if press_lower and press_upper:
        if 'Pressure p2' in doc.columns:
            test = doc[((doc['Pressure p1']>float(press_lower)) & (doc['Pressure p2'] > float(press_lower))) & ((doc['Pressure p1']<float(press_upper)) & (doc['Pressure p2'] < float(press_upper)))]
        else:
            test = doc[(doc['Abs. pressure p_abs'] < float(press_upper)) & (doc['Abs. pressure p_abs'] > float(press_lower))]
    else:
        press_lower = input('Please input lower limit of acceptable pressure range to look within. Default is 14.5')
        press_upper = input('Please input upper limit of acceptable pressure range to look within. Default is 14.7')
        if not press_lower:
            press_lower = 14.5
        if not press_upper:
            press_upper = 15.7
        if 'Pressure p2' in doc.columns:
            test = doc[((doc['Pressure p1']>float(press_lower)) & (doc['Pressure p2'] > float(press_lower))) & (doc['Pressure p1']<float(press_upper)) & (doc['Pressure p2'] < float(press_upper))]
        else:
            test = doc[(doc['Abs. pressure p_abs'] < float(press_upper)) & (doc['Abs. pressure p_abs'] > float(press_lower))]
    test.to_csv('Scrutiny_Files/Pressure Assortment Range.csv')
    return test
#The below looks for rows where the pressure 1 sensor is beyond a specified range.
def press_beyond_range(press_lower = None, press_upper = None, datatype = datatype_input):
    if press_lower and press_upper:
        if 'Pressure p2' in doc.columns:
            test = doc[(((doc['p1min']<float(press_lower)) | (doc['p1min'] > float(press_upper))) & ((doc['p2 min']<float(press_lower)) | (doc['p2 min'] > float(press_upper)))) | (((doc['p1max']<float(press_lower)) | (doc['p1max'] > float(press_upper))) & ((doc['p2 max']<float(press_lower)) | (doc['p2 max'] > float(press_upper))))]
        else:
            test = doc[((doc['p1min'] + 14.65 < float(press_lower)) | (doc['p1min']+14.65 > float(press_upper))) | ((doc['p1max']+14.65 < float(press_lower)) | (doc['p1max']+14.65 > float(press_upper)))]
    else:
        press_lower = input('Please input lower limit of acceptable pressure range to look within. Default is 14.5')
        press_upper = input('Please input upper limit of acceptable pressure range to look within. Default is 15.7')
        if not press_lower:
            press_lower = 14.5
        if not press_upper:
            press_upper = 15.7
        if 'Pressure p2' in doc.columns:
            test = doc[(((doc['p1min']<float(press_lower)) | (doc['p1min'] > float(press_upper))) & ((doc['p2 min']<float(press_lower)) | (doc['p2 min'] > float(press_upper)))) | (((doc['p1max']<float(press_lower)) | (doc['p1max'] > float(press_upper))) & ((doc['p2 max']<float(press_lower)) | (doc['p2 max'] > float(press_upper))))]
        else:
            test = doc[((doc['p1min'] + 14.65 < float(press_lower)) | (doc['p1min']+14.65 > float(press_upper))) | ((doc['p1max']+14.65 < float(press_lower)) | (doc['p1max']+14.65 > float(press_upper)))]
    test.to_csv('Scrutiny_Files/Pressure Beyond Range.csv')
    return test

# The below function uses the 'DateTime' column of the file to check for which minute values are not zero, possibly indicating disturbed time logs.
# It is for the below two functions that we converted the 'DateTime' from string type to datetime type.
#The below looks for rows where the time log is disturbed and not at a whole hour.
def disturbed_time_logs(datatype = datatype_input):
    empty_list = []
    for i in doc.index:
        if doc.at[i, 'DateTime'].minute != 0:
            empty_list.append(i)
    test = doc.loc[empty_list]
    test.to_csv('Scrutiny_Files/Disturbed Time Logs.csv')
    return test

# The below function makes use of the pandas offsets.DateOffset function to check if each time log is 1 hour after the last. If not, it may indicate missing time logs.
#The below looks for rows where there are missing time logs or the different between two time logs is no 0.
def missing_time_logs(datatype = datatype_input):
    empty_list = []
    for i in doc.index:
        if i == (len(doc.index)-5):
            doc.loc[empty_list].to_csv('Scrutiny_Files/Missing Time Logs.csv')
            return doc.loc[empty_list]
        if (doc.at[i+1, 'DateTime'] - pd.offsets.DateOffset(hours = 1)) != doc.at[i, "DateTime"]:
            if i not in empty_list:
                empty_list.append(i)
            empty_list.append(i+1)
    test = doc.loc[empty_list]
    test.to_csv('Scrutiny_Files/Missing Time Logs.csv')
    return test

# The below function checks for zero flow in variable period lengths from short (1 day), to continuous (2 days), to prolonged (3 days).
# This checks forward 24/48/72 rows ahead consecutively depending on the input to see if a period is characterized by continuous 0 flow.

def prolonged_zero_flow(period = 'prolonged', datatype = datatype_input):
    final_list = []
    empty_list = []
    if period:
        pass
    else:
        period = input("Please input the time range for which you would like to see zero flow values (E.g. short, continous, prolonged). Short = 1 day, continuous = > 1 day, prolonged = > 2 days. The default is short.")
    if not period:
        period = 'short'
    time = period.lower()
    if time == 'short':
        for i in doc.index:              
            if i == (len(doc.index)-25):
                return empty_list
            if any(i <= lst[1] for lst in empty_list):continue
            if doc.at[i,'Flow Q1'] == 0:
                for y in range(1,25):
                    if doc.at[i+y, 'Flow Q1'] == 0:
                        if y == 24:
                            empty_list.append([i,i+y])
                        continue
                    else:break
        return empty_list   
    if time == 'continuous':
        for i in doc.index:              
            if i == (len(doc.index)-49):
                return empty_list
            if any(i <= lst[1] for lst in empty_list):continue
            if doc.at[i,'Flow Q1'] == 0:
                for y in range(1,49):
                    if doc.at[i+y, 'Flow Q1'] == 0:
                        if y == 48:
                            empty_list.append([i,i+y])
                        continue
                    else:break
        return empty_list
    if time == 'prolonged':
        for i in doc.index:              
            if i == (len(doc.index)-73):
                return empty_list
            if any(i <= lst[1] for lst in empty_list):continue
            if doc.at[i,'Flow Q1'] == 0:
                for y in range(1,73):
                    if doc.at[i+y, 'Flow Q1'] == 0:
                        if y == 72:
                            empty_list.append([i,i+y])
                        continue
                    else:break
        return empty_list

# This function is an extension of the last and compiles the returned data into the proper format to slice the dataframe according to the conditions and save it as a csv.

def prolonged_zero_flow_u(period = 'prolonged', datatype = datatype_input):
    if not period:
        period = 'short'
    empty_lst = []
    for i in prolonged_zero_flow(period):
        empty_lst.append(doc.loc[i[0]:i[1]])
    test = pd.concat(empty_lst)
    test.to_csv('Scrutiny_Files/Prolonged Zero Flow.csv')
    return test

# The below two functions are almost identical to the ones above except for distinction: these check for reduced rather than zero flow, thus allowing for an input for a lower flow threshold.

def prolonged_reduced_flow(period = None, flow_lower = None, datatype = datatype_input):
    final_list = []
    empty_list = []
    if not flow_lower:
        flow_lower = input('Please input the lower flow limit to look below for prolonged/continuous periods. Default value is 200')
    if not flow_lower:
        flow_lower = float(200)
    if period:
        pass
    else:
        period = input("Please input the time range for which you would like to see zero flow values (E.g. short, continous, prolonged). Short = 1 day, continuous = > 1 day, prolonged = > 2 days. The default is short.")
    if not period:
        period = 'short'
    time = period.lower()
    if time == 'short':
        for i in doc.index:              
            if i == (len(doc.index)-25):
                return empty_list
            if any(i <= lst[1] for lst in empty_list):continue
            if doc.at[i,'Flow Q1'] < flow_lower:
                for y in range(1,25):
                    if doc.at[i+y, 'Flow Q1'] < float(flow_lower):
                        if y == 24:
                            empty_list.append([i,i+y])
                        continue
                    else:break
        return empty_list   
    if time == 'continuous':
        for i in doc.index:              
            if i == (len(doc.index)-49):
                return empty_list
            if any(i <= lst[1] for lst in empty_list):continue
            if doc.at[i,'Flow Q1'] < flow_lower:
                for y in range(1,49):
                    if doc.at[i+y, 'Flow Q1'] < float(flow_lower):
                        if y == 48:
                            empty_list.append([i,i+y])
                        continue
                    else:break
        return empty_list
    if time == 'prolonged':
        for i in doc.index:              
            if i == (len(doc.index)-73):
                return empty_list
            if any(i <= lst[1] for lst in empty_list):continue
            if doc.at[i,'Flow Q1'] < flow_lower:
                for y in range(1,73):
                    if doc.at[i+y, 'Flow Q1'] < float(flow_lower):
                        if y == 72:
                            empty_list.append([i,i+y])
                        continue
                    else:break
        return empty_list

def prolonged_reduced_flow_u(period = None, flow_lower = None, datatype = datatype_input):
    if not period:
        period = 'short'
    empty_lst = []
    for i in prolonged_reduced_flow(period, flow_lower):
        empty_lst.append(doc.loc[i[0]:i[1]])
    test = pd.concat(empty_lst)
    test.to_csv('Scrutiny_Files/Prolonged Reduced Flow.csv')
    return test
#The below looks for rows where there is an indication of a reset reading.
def reset_reading(minimum_threshhold_percentage = None, datatype = datatype_input):
    if not minimum_threshhold_percentage:
        minimum_threshhold_percentage = float(input('Please input a minimum threshhold percentage. Values between 0 and 1 are encouraged. The default value is 0.05. This will look for rows where the fault volume is less than 5% (using default value) of the previous fault volume.'))
    if not minimum_threshhold_percentage:
        minimum_threshhold_percentage == 0.05
    empty_list = []
    for i in doc.index:
        if i == int(len(doc.index)) -2 :
            return doc.loc[empty_list]
        else:
            if (doc.at[i+1, 'Primary volume V1'] <= minimum_threshhold_percentage*doc.at[i, 'Primary volume V1']) and (doc.at[i+1, 'Base volume Vb1'] <= minimum_threshhold_percentage*doc.at[i, 'Base volume Vb1']):
                empty_list.append(i+1)
    test = doc.loc[empty_list]
    test.csv('Scrutiny_Files/Reset Reading.csv')
    return doc.loc[empty_list]
#The below looks for rows where there is a fault in the columns pertaining to volume or where the increase between two rows for a volume column is significant.
def fault_volume(datatype = datatype_input):
    empty_list = []
    for i in doc.index:
        if i == int(len(doc.index)) -2 :
            return doc.loc[empty_list]
        else:
            if doc.at[i+1, 'Spare prim. vol. Vs1'] != doc.at[i, 'Spare prim. vol. Vs1']:
                empty_list.append(i+1)
    test = doc.loc[empty_list]
    test.csv('Scrutiny_Files/Fault Volume.csv')
    return doc.loc[empty_list]

In [ ]:
# The below is a list of names for the name column of the scrutiny table dataframe.

names = ['Zero p1 p2 Dual', 'Zero Min p1 p2 Dual', 'Dual Pressure Differential', 'Dual Pressure Differential Flow', 'Zero p1', 'Zero p1 Min', 'Zero p1 Flow', 'Zero p1 Min Flow', 'Zero p1/p1 Min & Flow', 'Flow Above Rated Capacity', 'Abnormal Press & Temp', 'Abnormal Flow', 'Reverse Flow', 'Reverse Flow & Zero Min/Abs Press', 'Battery Variations', 'Reg/Corr Volume Compat', 'Tamper Alarm & Zero Reduced Flow', 'Press Assortment Range', 'Pressure Beyond Range', 'Disturbed Time Logs', 'Missing Time Logs', 'Prolonged Zero Flow', 'Prolonged Reduced Flow', 'Reset Corrected/Uncorrected Reading', 'Uncorrected Fault Volume']
#The below uses the contents of the configfile to store the inputs for each function. This is so that the user can see in the compiled list what each function's inputted parameters were.
inputs = ['Dual Lower Pressure Limit: ' + str(config.get('zero_p1_p2_dual','pressure_input')), 'Dual Lower Minimum Pressure Limit: ' + str(config.get('zero_min_p1_p2_dual','pressure_input')), 'Dual Pressure Differential: ' + str(config.get('dual_pressure_differential','pressure_differential_input')), 
'Dual Pressure Differential Flow Limit: ' + str(config.get('dual_pressure_diff_flow','pressure_differential_input')) + ', Dual Pressure Lower: ' + str(config.get('dual_pressure_diff_flow','flow_lower')) + ', Dual Pressure Upper: ' +  str(config.get('dual_pressure_diff_flow','flow_upper')), 
'Pressure Lower Limit: ' + str(config.get('zero_p1','pressure_input')), 'Minimum Pressure Lower Limit: ' + str(config.get('zero_p1min', 'minimum_pressure_input')), 'Zero Pressure Lower Limit: ' + str(config.get('zero_p1_flow','pressure_input')), 
'Zero Minimum Pressure Limit: ' + str(config.get('zero_p1_min_flow', 'minimum_pressure_input')), 'Zero Min/Abs Pressure Limit: ' + str(config.get('zero_p1_or_p1_min_flow', 'minimum_pressure_input')), 'Rated Capacity Upper Limit: ' + str(config.get('flow_above_rated_capacity', 'rated_capacity')), 
'Temperature Lower Limit: ' + str(config.get('abnormal_pressure_and_temp', 'temp_lower')) + ', Temperature Upper Limit: ' + str(config.get('abnormal_pressure_and_temp', 'temp_lower')) + ', Pressure Lower Limit: ' + str(config.get('abnormal_pressure_and_temp', 'press_input')),
'Abnormal Flow Lower Limit: ' + str(config.get('abnormal_flow', 'flow_lower')) + ', Abnormal Flow Upper Limit: ' + str(config.get('abnormal_flow', 'flow_upper')), 'No Input Required', 'Minimum Pressure Lower Limit: ' + str(config.get('rev_flow_and_zero_abs_or_min_pressure', 
'minimum_pressure_input')), 'Battery Lower Limit: ' + str(config.get('battery_variations', 'battery_lower')), 'Compatibility Ratio: ' + str(config.get('reg_corr_volume_compat', 'compatibility_ratio')), 'Flow Lower Limit: ' + str(config.get('tamper_alarm_and_zero_or_reduced_flow', 'flow_lower')),
'Pressure Lower Limit: ' + str(config.get('pressure_assortment_range', 'press_lower')) + ', Pressure Upper Limit: ' + str(config.get('pressure_assortment_range', 'press_upper')), 'Pressure Beyond Lower Limit: ' + str(config.get('press_beyond_range', 'press_lower')) + ', Pressure Beyond Upper Limit: ' + str(config.get('press_beyond_range', 'press_upper')), 
'No Input Required', 'No Input Required', 'Period: ' + str(config.get('prolonged_zero_flow', 'period')), 'Period: ' + str(config.get('prolonged_reduced_flow', 'period')) + ', Lower Flow Limit: ' + str(config.get('prolonged_reduced_flow', 'flow_lower')), 'Minimum Threshhold Percentage: ' + str(config.get('reset_reading', 'minimum_threshhold_percentage')) + '%', 'No Input Required']
# The below is a function which returns the lengths of the scrutiny functions as a list. It includes specified arguments to skip the need for inputs or output.

#The below calls all functions and stores the length of the results of each function in a list.
def config_count():
    return [len(zero_p1(float(config.get('zero_p1','pressure_input')))), len(zero_p1min(float(config.get('zero_p1min', 'minimum_pressure_input')))), len(zero_p1_flow(float(config.get('zero_p1_flow','pressure_input')))), len(zero_p1min_flow(float(config.get('zero_p1_min_flow', 'minimum_pressure_input')))), 
    len(zero_p1_or_p1min_and_flow(float(config.get('zero_p1_or_p1_min_flow', 'minimum_pressure_input')))), len(flow_above_rated_capacity(float(config.get('flow_above_rated_capacity', 'rated_capacity')))), len(abnormal_press_and_temp(float(config.get('abnormal_pressure_and_temp', 'temp_lower')), float(config.get('abnormal_pressure_and_temp', 'temp_lower')), 15)),
    len(abnormal_flow(float(config.get('abnormal_flow', 'flow_lower')), float(config.get('abnormal_flow', 'flow_upper')))), len(reverse_flow()), len(accum_reverse_flow()), len(rev_flow_and_zero_min_or_abs_press(float(config.get('rev_flow_and_zero_abs_or_min_pressure', 'minimum_pressure_input')))), 
    len(battery_variations(float(config.get('battery_variations', 'battery_lower')))), len(reg_corr_volume_compat(float(config.get('reg_corr_volume_compat', 'compatibility_ratio')))), len(tamper_alarm_and_zero_reduced_flow(float(config.get('tamper_alarm_and_zero_or_reduced_flow', 'flow_lower')))),
    len(press_assortment_range(float(config.get('pressure_assortment_range', 'press_lower')), float(config.get('pressure_assortment_range', 'press_upper')))), len(disturbed_time_logs()), len(missing_time_logs()), len(prolonged_zero_flow_u(config.get('prolonged_zero_flow', 'period'))), len(prolonged_reduced_flow_u(config.get('prolonged_reduced_flow', 'period')))]
#The below uses the contents of the configfile to call each function and store the results in a list in the return statement of this function.
def config_count_u():
    return [(zero_p1_p2_dual(float(config.get('zero_p1_p2_dual','pressure_input')))), (zero_min_p1_p2_dual(float(config.get('zero_min_p1_p2_dual','pressure_input')))), (dual_pressure_differential(float(config.get('dual_pressure_differential','pressure_differential_input')))), 
    (dual_pressure_diff_flow(float(config.get('dual_pressure_diff_flow','pressure_differential_input')), float(config.get('dual_pressure_diff_flow','flow_lower')), float(config.get('dual_pressure_diff_flow','flow_upper')))),(zero_p1(float(config.get('zero_p1','pressure_input')))), (zero_p1min(float(config.get('zero_p1min', 'minimum_pressure_input')))),
    (zero_p1_flow(float(config.get('zero_p1_flow','pressure_input')))), (zero_p1min_flow(float(config.get('zero_p1_min_flow', 'minimum_pressure_input')))), (zero_p1_or_p1min_and_flow(float(config.get('zero_p1_or_p1_min_flow', 'minimum_pressure_input')))), (flow_above_rated_capacity(float(config.get('flow_above_rated_capacity', 'rated_capacity')))),
    (abnormal_press_and_temp(float(config.get('abnormal_pressure_and_temp', 'temp_lower')), float(config.get('abnormal_pressure_and_temp', 'temp_lower')), 15)),
    (abnormal_flow(float(config.get('abnormal_flow', 'flow_lower')), float(config.get('abnormal_flow', 'flow_upper')))), (reverse_flow()), (rev_flow_and_zero_min_or_abs_press(float(config.get('rev_flow_and_zero_abs_or_min_pressure', 'minimum_pressure_input')))),
    (battery_variations(float(config.get('battery_variations', 'battery_lower')))), (reg_corr_volume_compat(float(config.get('reg_corr_volume_compat', 'compatibility_ratio')))), (tamper_alarm_and_zero_reduced_flow(float(config.get('tamper_alarm_and_zero_or_reduced_flow', 'flow_lower')))),
    (press_assortment_range(float(config.get('pressure_assortment_range', 'press_lower')), float(config.get('pressure_assortment_range', 'press_upper')))), (press_beyond_range(float(config.get('press_beyond_range', 'press_lower')), float(config.get('press_beyond_range', 'press_upper')))), 
    (disturbed_time_logs()), (missing_time_logs()), (prolonged_zero_flow_u(config.get('prolonged_zero_flow', 'period'))), (prolonged_reduced_flow_u(config.get('prolonged_reduced_flow', 'period'), float(config.get('prolonged_reduced_flow', 'flow_lower')))), reset_reading(float(config.get('reset_reading', 'minimum_threshhold_percentage'))), fault_volume()]


In [ ]:
# listry = []
# for i in config_count_u():
#     if len(i) > 0:
#         listry.append(pd.to_datetime(i.iloc[0,0]))
#     else:
#         listry.append('No Data')


#The below compiles all the results from the function above.
lister = []
for i in config_count_u():
    if type(i) == str:
        lister.append('No Dual Sensor Data')
    else:
        lister.append(len(i))
lister
testry = []
#The below takes the name of each functions, the length of each function's output, and the inputs of each function. After combining it into a list, it makes a pd datafram out of it and saves it as a csv.
for i in range(len(names)):
    testry += [[names[i], lister[i], inputs[i]]] #Add listry[i] for first occurrence

df = pd.DataFrame(testry, columns=['Scrutiny Point', 'Count', 'Inputs']) #Add first occurrence column
df.index += 1

df.to_csv('Scrutiny_Files/Scrutiny Table.csv')
